In [8]:
#################################
# IMPORT THE REQUIRED LIBRARIES #
#################################
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
import regex
import numpy as np
import datetime as dt
import dateutil.relativedelta
import statsmodels.api as sm
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils import class_weight

from numpy.random import seed
from keras.callbacks import EarlyStopping

seed(1)
import tensorflow as tf

from sklearn.preprocessing import MinMaxScaler

tf.random.set_seed(1234)
rnd = np.random.RandomState(1)

from warnings import filterwarnings
filterwarnings("ignore")

%matplotlib inline

import sklearn
from numpy import interp
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, roc_curve
from sklearn.model_selection import KFold
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import accuracy_score

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from mord import LogisticAT
from sklearn.svm import LinearSVC, NuSVC, SVC
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression, SGDClassifier
from sklearn.ensemble import BaggingClassifier, ExtraTreesClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense, Dropout
import keras

from numpy import interp
from sklearn.metrics import auc, average_precision_score, confusion_matrix, roc_curve, precision_recall_curve
from sklearn.model_selection import KFold, train_test_split, RandomizedSearchCV, StratifiedKFold


from sklearn.metrics import auc, f1_score
from sklearn.metrics import plot_roc_curve
from sklearn.model_selection import StratifiedKFold


In [7]:
# pip install mord

In [ ]:
####################
# READ THE DATASET #
####################

df = pd.read_csv('../input/strokedatasetweather/stroke_data.csv')

##########################
# EDIT THE DATA FORMAT   #
##########################

In [11]:
df.head()

,gender,age,diabetes,hypertension,stroke,heart disease,smoking history,BMI
0,Female,80.0,0,0,0,1,never,25.19
1,Female,54.0,0,0,0,0,NaN,NaN
2,Male,28.0,0,0,0,0,never,NaN
3,Female,36.0,0,0,0,0,current,23.45
4,Male,76.0,0,1,0,1,current,20.14


In [12]:
df['ONSET_Date'] = pd.to_datetime(df['ONSET_Date'])
df.Discharge_Plan_Modified_Rankin_Score = pd.to_numeric(df.Discharge_Plan_Modified_Rankin_Score, errors='coerce')

KeyError: 'ONSET_Date'

In [13]:




##################################################################################
# ADD ONE MORE PREDICTOR - a derivative variable called "Aterogenic Index" - IA) #
# REMOVE INCORRECT VALUES                                                        #
##################################################################################

df['IA'] = df['Lab_Investigation_low-density_lipoprotein']/(df['Lab_Investigation_TotalCholeserol']-df['Lab_Investigation_low-density_lipoprotein'])

temp = df[(df['Lab_Investigation_low-density_lipoprotein'].isna())|(df['Lab_Investigation_TotalCholeserol'].isna())]
temp = temp[['Lab_Investigation_low-density_lipoprotein','Lab_Investigation_TotalCholeserol']]
ind1 = temp[~temp.iloc[:,0].isna()].index 
ind2 = temp[~temp.iloc[:,1].isna()].index
ai_ind = ind1.append(ind2)

for i in ai_ind:
    df.loc[i,'IA'] = np.nan
    
df['IA'].replace([np.inf, -np.inf], np.nan,inplace=True)

#########################
# ENCODE TIME INTERVALS #
#########################

df['Day_Time'].replace({'Evening':1},inplace = True)
df['Day_Time'].replace({'Afternoon':2},inplace = True)
df['Day_Time'].replace({'Morning':3},inplace = True)
df['Day_Time'].replace({'Night':4},inplace = True)

###############################
# EDIT MISTYPING ERRORS       #
###############################

ind = df[df['Lab_Investigation_TotalCholeserol']==4347].index
df.loc[ind,'Lab_Investigation_TotalCholeserol'] = 4.347

ind1 = df[df['Lab_Investigation_low-density_lipoprotein']==3157].index
df.loc[ind1,'Lab_Investigation_low-density_lipoprotein'] = 3.157

KeyError: 'Lab_Investigation_low-density_lipoprotein'

In [2]:
data = df[df['ICH']==True].copy()

data.drop(columns=['ICH','Lab_Investigation_international_norm_ratio',
       'Lab_Investigation_C-reactive protein',
       'Lab_Investigation_TotalCholeserol',
       'Lab_Investigation_low-density_lipoprotein',
       'Lab_Investigation_POC_Random blood sugar',
       'Lab_Investigation_Creatinine','IA','ONSET_Date'],inplace = True)

NameError: name 'df' is not defined

In [31]:
#########################################################################
# 1.PROGNOSTICATION OF HS SEVERITY (NIHSS group) FROM INDIVIDUAL RISKS  #
#########################################################################

In [89]:
data_nihss = data[~data.NIHSS_group.isna()]
data_nihss['NIHSS_group'] = np.where(data_nihss['Screening_tools_NIHSS']<=4,0,1)
data_nihss['NIHSS_group'].value_counts()

1    47
0    38
Name: NIHSS_group, dtype: int64

In [90]:
def build_barplot_nihss(df,feature,plot_title,xlabel):
    
    fig = plt.figure(figsize=(8,8))

    g = sns.countplot(df[feature].replace({0:'No',1:'Yes'}), hue = df['NIHSS_group'])
    plt.title(plot_title)
    plt.xlabel(xlabel)
    plt.ylabel('Counts')

    leg = g.axes.get_legend()
    new_title = 'NIHSS score'
    leg.set_title(new_title)
    new_labels = ['Less or equal to 4', 'More than 4']
    for t, l in zip(leg.texts, new_labels): t.set_text(l)

    for p in g.patches:
        g.annotate('{:.0f}'.format(p.get_height()), (p.get_x()+0.15, p.get_height()+0.5))

    fig.show()
    fig.savefig('ICH-NIHSS_{}_barplot.png'.format(feature))

In [ ]:
build_barplot_nihss(data_nihss,'History_OldStroke','NIHSS group vs Old Stroke','Old Stroke')
build_barplot_nihss(data_nihss,'History_DM','NIHSS group vs Diabetes Mellitus','Diabetes Mellitus')
build_barplot_nihss(data_nihss,'History_HyperTension','NIHSS group vs Hyper Tension','Hyper Tension')
build_barplot_nihss(data_nihss,'History_IschemicHeartDisease','NIHSS group vs Ischemic Heart Disease','Ischemic Heart Disease')
build_barplot_nihss(data_nihss,'History_ArterFibrillation','NIHSS group vs Arter Fibrillation','Arter Fibrillation')
build_barplot_nihss(data_nihss,'History_HyperLypidAemia','NIHSS group vs Hyper Lypid Aemia','Hyper Lypid Aemia')

fig = plt.figure(figsize=(8,8))

g = sns.countplot(data_nihss['Day_Time'].replace({1:'Evening',2:'Afternoon',3:'Morning',4:'Night'}), hue = data_nihss['NIHSS_group'])
plt.title('NIHSS vs Day Time')
plt.xlabel('History: Day Time')
plt.ylabel('Counts')

leg = g.axes.get_legend()
new_title = 'NIHSS score'
leg.set_title(new_title)
new_labels = ['Less or equal to 4', 'More than 4']
for t, l in zip(leg.texts, new_labels): t.set_text(l)
     
for p in g.patches:
    g.annotate('{:.0f}'.format(p.get_height()), (p.get_x()+0.15, p.get_height()+0.2))
    
fig.show()
fig.savefig('ICH-NIHSS_DAY_TIME_barplot.png')

In [92]:
weather_cols = ['TEMP', 'STP', 'WDSP', 'RH', 'HUMIDEX', 'TEMP1', 'TEMP2', 'TEMP3', 'TEMP4', 'TEMP5',
                'TEMP6', 'TEMP7', 'STP1', 'STP2', 'STP3', 'STP4', 'STP5', 'STP6', 'STP7', 'WDSP1',
                'WDSP2', 'WDSP3', 'WDSP4', 'WDSP5', 'WDSP6', 'WDSP7', 'RH1', 'RH2', 'RH3', 'RH4',
                'RH5', 'RH6', 'RH7', 'HUMIDEX1', 'HUMIDEX2', 'HUMIDEX3', 'HUMIDEX4', 'HUMIDEX5',
                'HUMIDEX6', 'HUMIDEX7', 'TDIF1', 'TDIF2', 'TDIF3', 'TDIF4', 'TDIF5', 'TDIF6', 'TDIF7',
                'PDIF1', 'PDIF2', 'PDIF3', 'PDIF4', 'PDIF5', 'PDIF6', 'PDIF7', 'WDIF1', 'WDIF2', 'WDIF3',
                'WDIF4', 'WDIF5', 'WDIF6', 'WDIF7', 'RHDIF1', 'RHDIF2', 'RHDIF3', 'RHDIF4', 'RHDIF5',
                'RHDIF6', 'RHDIF7', 'HDIF1', 'HDIF2', 'HDIF3', 'HDIF4', 'HDIF5', 'HDIF6', 'HDIF7']

non_weather_cols = data_nihss.columns.difference(weather_cols)

data_nihss_nw = data_nihss[non_weather_cols]

In [93]:
def get_outliers(df,col):
    q1 = df[col].quantile(0.25) 
    q3 = df[col].quantile(0.75) 
    iqr = q3-q1
    
    return df[col][(df[col] > q3 + 1.5*iqr)|(df[col] < q1 - 1.5*iqr)].index

In [94]:
out = []
for col in ['BMI', 'DEMOGRAPHY_age','Day_Time']:
    l = get_outliers(data_nihss_nw ,col)
    print(col, l)
out

BMI Int64Index([1125, 1150, 1153, 1182, 1196, 1255, 1263, 1275], dtype='int64')

DEMOGRAPHY_age Int64Index([], dtype='int64')

Day_Time Int64Index([], dtype='int64')


[]

In [95]:
ind_bmi = [1125, 1150, 1153, 1182, 1196, 1255, 1263, 1275]

In [96]:
data_nihss_nw.columns

Index(['BMI', 'DEMOGRAPHY_age', 'DEMOGRAPHY_sex', 'Day_Time',
       'Discharge_Plan_Modified_Rankin_Score', 'History_ArterFibrillation',
       'History_DM', 'History_HyperLypidAemia', 'History_HyperTension',
       'History_IschemicHeartDisease', 'History_OldStroke', 'History_Smoking',
       'NIHSS_group', 'Screening_tools_NIHSS'],
      dtype='object')

In [97]:
def get_weights(X,Y,cat=True):
    ada = AdaBoostClassifier()
    gb = GradientBoostingClassifier()
    rf = RandomForestClassifier()
    et = ExtraTreesClassifier()
    
    X_values = preprocessor.fit_transform(X)
    
    if (cat == True):
        onehot_columns = list(preprocessor.transformers_[1][1]['onehot'].get_feature_names(cat_col))
        numeric_features_list = list(num_col)
        numeric_features_list.extend(onehot_columns)
    else:
        numeric_features_list = list(num_col)
        
    df_from_array_pipeline = pd.DataFrame(X_values, columns = numeric_features_list)
    
    ada.fit(X_values,Y)
    gb.fit(X_values,Y)
    rf.fit(X_values,Y)
    et.fit(X_values,Y)

    df_ada = pd.DataFrame({'feature':np.array(numeric_features_list), 'importance' : ada.feature_importances_})
    df_gb = pd.DataFrame({'feature':np.array(numeric_features_list), 'importance' : gb.feature_importances_})
    df_rf = pd.DataFrame({'feature':np.array(numeric_features_list), 'importance' : rf.feature_importances_})
    df_et = pd.DataFrame({'feature':np.array(numeric_features_list), 'importance' : et.feature_importances_})
    
    result = pd.merge(df_ada[['feature','importance']], df_gb[['feature','importance']], on='feature')
    result = pd.merge(result, df_rf[['feature','importance']], on='feature')
    result = pd.merge(result, df_et[['feature','importance']], on='feature')
    result['importance'] = result.mean(axis=1)
    result = result.sort_values(by='importance', ascending = False)

    return result[['feature','importance']]

In [98]:
def draw_cv_roc_curve(classifiers, cv, X, y, title, filename, weights = False):
    """
    Draw a Cross Validated PR Curve.
    Keyword Args:
        classifier: Classifier Object
        cv: StratifiedKFold Object: (https://stats.stackexchange.com/questions/49540/understanding-stratified-cross-validation)
        X: Feature Pandas DataFrame
        y: Response Pandas Series

    Largely taken from: https://stackoverflow.com/questions/29656550/how-to-plot-pr-curve-over-10-folds-of-cross-validation-in-scikit-learn
    """
    
    fig1 = plt.figure(figsize=[9,9])
    ax1 = fig1.add_subplot(111,aspect = 'equal')

    for clf in classifiers:
        cv = StratifiedKFold(n_splits=10)
        tprs = []
        aucs = []

        mean_fpr = np.linspace(0,1,100)
        i = 1
        for train, test in cv.split(X,y):
            prediction = clf.fit(X[train],y[train]).predict_proba(X[test])

            fpr, tpr, t = roc_curve(y[test], prediction[:, 1])
            tprs.append(interp(mean_fpr, fpr, tpr))
            roc_auc = auc(fpr, tpr)
            aucs.append(roc_auc)
            i= i+1

        plt.plot([0,1],[0,1],linestyle = '--',lw = 2,color = 'black')
        mean_tpr = np.mean(tprs, axis=0)
        mean_auc = auc(mean_fpr, mean_tpr)
        plt.plot(mean_fpr, mean_tpr, color=np.random.rand(3,),
                 label=r'{} Mean ROC (AUC = {:.3} )'.format(clf.__class__.__name__,mean_auc),lw=2, alpha=1)
        
    model = Sequential()
    model.add(Dense(12, input_dim = len(X[0]), activation = 'relu'))
    model.add(Dense(10, activation = 'relu'))
    model.add(Dense(1, activation = 'sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer = 'adam')        
    
    class_weights = class_weight.compute_class_weight(
               'balanced',
                np.unique(y), 
                y)

    class_dict = {0:class_weights[0],1:class_weights[1]} 
            
    for train, test in cv.split(X, y):
        es = EarlyStopping(monitor='val_loss', mode='min', verbose=0)
        
        if (weights == True):
            model.fit(X[train],y[train],validation_data=(X[test],y[test]),class_weight=class_dict, verbose=0,epochs=500)
        else:
            model.fit(X[train],y[train],validation_data=(X[test],y[test]),verbose=0,epochs=500)
            
        prediction = model.predict(X[test])
        
        fpr, tpr, t = roc_curve(y[test], prediction)
        tprs.append(interp(mean_fpr, fpr, tpr))
        roc_auc = auc(fpr, tpr)
        aucs.append(roc_auc)
        #  plt.plot(fpr, tpr, lw=2, alpha=0.3, label='ROC fold %d (AUC = %0.2f)' % (i, roc_auc))
        i= i+1

    plt.plot([0,1],[0,1],linestyle = '--',lw = 2,color = 'black')
    mean_tpr = np.mean(tprs, axis=0)
    mean_auc = auc(mean_fpr, mean_tpr)
    plt.plot(mean_fpr, mean_tpr, color=np.random.rand(3,),
             label=r'{} Mean ROC (AUC = {:.3} )'.format('Neural Network',mean_auc),lw=2, alpha=1)
    
    plt.xlim([-0.05, 1.05])
    plt.ylim([-0.05, 1.05])
    
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC')
    plt.legend(loc="lower right")
   
    plt.show()
    fig1.savefig('ICH-{}.png'.format(filename))

In [99]:
def df_cat_nihss(df, scaler, outliers=False, weights=False):

    X = data_nihss_nw.drop(columns=['NIHSS_group','Discharge_Plan_Modified_Rankin_Score','Screening_tools_NIHSS'])
    Y = data_nihss_nw['NIHSS_group']
    
    if (outliers == True):
        X.drop(index = [1125, 1150, 1153, 1182, 1196, 1255, 1263, 1275], inplace = True)
        Y.drop(index = [1125, 1150, 1153, 1182, 1196, 1255, 1263, 1275], inplace = True)    
    
    cat_col = ['DEMOGRAPHY_sex','History_ArterFibrillation',
               'History_DM','History_HyperLypidAemia','History_HyperTension','History_IschemicHeartDisease',
               'History_OldStroke','History_Smoking']

    num_col = X.columns.difference(cat_col)
    
    if scaler == 'standard':
        sclr = StandardScaler()
    else:
        sclr = MinMaxScaler()
        
        
    numerical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', sclr),
    ])

    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(drop='first')),
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, num_col),
            ('cat', categorical_transformer, cat_col)
    ])
    print(cat_col)
    print(X.columns)
    
    ada = AdaBoostClassifier()
    gb = GradientBoostingClassifier()
    rf = RandomForestClassifier()
    et = ExtraTreesClassifier()
    
    X_values = preprocessor.fit_transform(X)
    
    onehot_columns = list(preprocessor.transformers_[1][1]['onehot'].get_feature_names(cat_col))
    numeric_features_list = list(num_col)
    numeric_features_list.extend(onehot_columns)
        
    df_from_array_pipeline = pd.DataFrame(X_values, columns = numeric_features_list)
    
    ada.fit(X_values,Y)
    gb.fit(X_values,Y)
    rf.fit(X_values,Y)
    et.fit(X_values,Y)

    df_ada = pd.DataFrame({'feature':np.array(numeric_features_list), 'importance' : ada.feature_importances_})
    df_gb = pd.DataFrame({'feature':np.array(numeric_features_list), 'importance' : gb.feature_importances_})
    df_rf = pd.DataFrame({'feature':np.array(numeric_features_list), 'importance' : rf.feature_importances_})
    df_et = pd.DataFrame({'feature':np.array(numeric_features_list), 'importance' : et.feature_importances_})
    
    result = pd.merge(df_ada[['feature','importance']], df_gb[['feature','importance']], on='feature')
    result = pd.merge(result, df_rf[['feature','importance']], on='feature')
    result = pd.merge(result, df_et[['feature','importance']], on='feature')
    result['importance'] = result.mean(axis=1)
    result = result.sort_values(by='importance', ascending = False)
    feature_importances = result[['feature','importance']]
    print(feature_importances.to_latex())
    
    fig = plt.figure(figsize=(15,6))

    g = sns.barplot(x='feature',y='importance',data=feature_importances, palette = 'tab10')
    g.set_xticklabels(rotation=90,labels = feature_importances['feature'])
    g.set_title('Non-weather feature importances for NIHSS group')
    plt.show()
    
            
    if (weights == True):
        w_str = 'weights'
    else:
        w_str = 'NOweights'
        
    if (outliers == True):
        o_str = 'out'
    else:
        o_str = 'NOout'
        
    file_str = 'ICH-NIHSS-FI-NW-{}-{}-{}.png'.format(o_str,w_str,scaler)    
    
    feature_importances.to_csv('ICH-NIHSS-FI-NW-{}-{}-{}.csv'.format(o_str,w_str,scaler))
    
    fig.savefig(file_str)

    to_keep = feature_importances.feature[:3]#['BMI', 'DEMOGRAPHY_age', 'Day_Time']
    X = data_nihss_nw[to_keep]
    Y = data_nihss_nw['NIHSS_group']
    
    
    if (outliers == True):
        X.drop(index = [1125, 1150, 1153, 1182, 1196, 1255, 1263, 1275], inplace = True)
        Y.drop(index = [1125, 1150, 1153, 1182, 1196, 1255, 1263, 1275], inplace = True) 
        
    cat_col = []

    num_col = X.columns.difference(cat_col)

    numerical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', sclr),
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, num_col),
    ])


    models = [
        LogisticRegression(solver='lbfgs'), 
        RandomForestClassifier(n_estimators=300),
        ExtraTreesClassifier(n_estimators=300),
        AdaBoostClassifier(n_estimators=300),
        SVC(kernel="linear", C=0.025, probability = True)    
    ]

    X = preprocessor.fit_transform(X)
    Y = np.array(Y)    
        
    cv = StratifiedKFold(n_splits=10,random_state=rnd)
    
    if (weights == True):
        w_str = 'weights'
    else:
        w_str = 'NOweights'
        
    if (outliers == True):
        o_str = 'out'
    else:
        o_str = 'NOout'
        
    file_str = 'NIHSS-ROC-NW-{}-{}-{}.png'.format(o_str,w_str,scaler)    
        
    draw_cv_roc_curve(models, cv, X, Y, title='ROC for NIHSS Group Prediction (non-weather predictors)',filename = file_str, weights = weights)

In [ ]:
print('STANDARD SCALER, no weights')
df_cat_nihss(data_nihss_nw, 'standard', weights  = False, outliers  = False)
print('\n')

print('STANDARD SCALER, weights')
df_cat_nihss(data_nihss_nw, 'standard', weights  = True, outliers  = False)
print('\n')

print('STANDARD SCALER, no outliers')
df_cat_nihss(data_nihss_nw, 'standard', weights  = False, outliers  = True)
print('\n')

print('STANDARD SCALER, weights, no outliers')
df_cat_nihss(data_nihss_nw, 'standard', weights  = True, outliers  = True)
print('\n')

print('MINMAX SCALER, no weights')
df_cat_nihss(data_nihss_nw, 'minmax', weights  = False, outliers  = False)
print('\n')

print('MINMAX SCALER, weights')
df_cat_nihss(data_nihss_nw, 'minmax', weights  = True, outliers  = False)
print('\n')

print('MINMAX SCALER, no outliers')
df_cat_nihss(data_nihss_nw, 'minmax', weights  = False, outliers  = True)
print('\n')

print('MINMAX SCALER, weights, no outliers')
df_cat_nihss(data_nihss_nw, 'minmax', weights  = True, outliers  = True)
print('\n')

In [101]:
print(MinMaxScaler())

MinMaxScaler()


In [ ]:
##################################################################################
# 2.PORGNOSTICATION OF HS SEVERITY (NIHSS group) FROM THE FULL SET OF PREDICTORS #
# (BOTH INDIVIDUAL RISKS AND WEATHER PARAMETERS)                                 #
##################################################################################

In [ ]:
out = []
temp = data_nihss[['DEMOGRAPHY_age', 'BMI', 'TEMP', 'STP', 'WDSP', 'RH', 'HUMIDEX','Day_Time',
       'TEMP1', 'TEMP2', 'STP1', 'STP2', 'WDSP1', 'WDSP2', 'RH1', 'RH2',
       'HUMIDEX1', 'HUMIDEX2', 'TDIF1', 'TDIF2', 'PDIF1', 'PDIF2', 'WDIF1',
       'WDIF2', 'RHDIF1', 'RHDIF2', 'HDIF1', 'HDIF2', 'NIHSS_group']]
for col in temp.columns:
    l = get_outliers(temp,col)
    print(col, l)
out

In [103]:
ind_bmi = [1125, 1150, 1153, 1182, 1196, 1255, 1263, 1275]
ind_wdsp = [1182, 1207, 1229, 1265]
ind_wdsp1 = [1123, 1229, 1259]
ind_wdsp2 = [1259, 1260]
ind_tdif1 = [1259, 1265]
ind_tdif2 = [1123, 1161, 1187, 1249, 1256, 1260, 1264]
ind_pdif1 = [1123, 1265]
ind_pdif2 = [1123, 1257]
ind_wdif1 = [1123]
ind_wdif2 = [1120, 1122, 1123, 1185, 1256]
ind_rhdif1 = [1146]
ind_rhdif2 = [1187, 1249, 1250, 1260, 1264]
ind_hdif1 = [1120, 1146, 1259]
ind_hdif2 = [1123, 1141, 1149, 1256, 1264]

In [104]:
ind_out = ind_bmi + ind_wdsp + ind_wdsp1 + ind_wdsp2 + ind_tdif1 + ind_tdif2 +ind_pdif1 + ind_pdif2 + ind_wdif1 + ind_wdif2 + ind_rhdif1 + ind_rhdif2 + ind_hdif1 + ind_hdif2
ind_out = pd.DataFrame(pd.Series(ind_out).value_counts())
ind_out = ind_out[ind_out.iloc[:,0]>1].index
print(ind_out)

Int64Index([1123, 1259, 1256, 1264, 1265, 1260, 1229, 1146, 1187, 1182, 1120,

            1249],

           dtype='int64')


In [105]:
def df_all_nihss(df, scaler, outliers=False, weights=False):
    data_nihss_all = data_nihss[['DEMOGRAPHY_age', 'BMI', 'TEMP', 'STP', 'WDSP', 'RH', 'HUMIDEX','Day_Time',
       'TEMP1', 'TEMP2', 'STP1', 'STP2', 'WDSP1', 'WDSP2', 'RH1', 'RH2',
       'HUMIDEX1', 'HUMIDEX2', 'TDIF1', 'TDIF2', 'PDIF1', 'PDIF2', 'WDIF1',
       'WDIF2', 'RHDIF1', 'RHDIF2', 'HDIF1', 'HDIF2', 'NIHSS_group']]
    cor_matrix = data_nihss_all.corr().abs()
    upper_tri = cor_matrix.where(np.triu(np.ones(cor_matrix.shape),k=1).astype(np.bool))
    to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.95)]
    data_nihss_all = data_nihss_all.drop(columns = to_drop)


    
    X = data_nihss_all.drop(columns=['NIHSS_group'])
    Y = data_nihss_all['NIHSS_group']

    if (outliers == True):
        X.drop(index = [1123, 1259, 1256, 1264, 1265, 1260, 1229, 1146, 1187, 1182, 1120,1249], inplace = True)
        Y.drop(index = [1123, 1259, 1256, 1264, 1265, 1260, 1229, 1146, 1187, 1182, 1120,1249], inplace = True)    

    cat_col = []

    num_col = X.columns.difference(cat_col)

    if scaler == 'standard':
        sclr = StandardScaler()
    else:
        sclr = MinMaxScaler()

    
    numerical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', sclr),
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, num_col),
    ])

    ada = AdaBoostClassifier()
    gb = GradientBoostingClassifier()
    rf = RandomForestClassifier()
    et = ExtraTreesClassifier()
    
    X_values = preprocessor.fit_transform(X)
    
    numeric_features_list = list(num_col)
    
    df_from_array_pipeline = pd.DataFrame(X_values, columns = numeric_features_list)
    
    ada.fit(X_values,Y)
    gb.fit(X_values,Y)
    rf.fit(X_values,Y)
    et.fit(X_values,Y)

    df_ada = pd.DataFrame({'feature':np.array(numeric_features_list), 'importance' : ada.feature_importances_})
    df_gb = pd.DataFrame({'feature':np.array(numeric_features_list), 'importance' : gb.feature_importances_})
    df_rf = pd.DataFrame({'feature':np.array(numeric_features_list), 'importance' : rf.feature_importances_})
    df_et = pd.DataFrame({'feature':np.array(numeric_features_list), 'importance' : et.feature_importances_})
    
    result = pd.merge(df_ada[['feature','importance']], df_gb[['feature','importance']], on='feature')
    result = pd.merge(result, df_rf[['feature','importance']], on='feature')
    result = pd.merge(result, df_et[['feature','importance']], on='feature')
    result['importance'] = result.mean(axis=1)
    result = result.sort_values(by='importance', ascending = False)
    feature_importances = result[['feature','importance']]
    print(feature_importances.to_latex())
    to_keep = list(feature_importances[feature_importances['importance']>0.04]['feature'])
    print('Features to keep:', to_keep)
    X = data_nihss_all[to_keep]
    
   # fig, axs = plt.subplots(1,2)
    
    fig = plt.figure(figsize=(15,6))

    g = sns.barplot(x='feature',y='importance',data=feature_importances, palette = 'tab10')
    g.set_xticklabels(rotation=90,labels = feature_importances['feature'])
    g.set_title('All feature importances for NIHSS group')
    plt.show()
    
        
    if (weights == True):
        w_str = 'weights'
    else:
        w_str = 'NOweights'
        
    if (outliers == True):
        o_str = 'out'
    else:
        o_str = 'NOout'
        
    file_str = 'ICH-NIHSS-FI-ALL-{}-{}-{}.png'.format(o_str,w_str,scaler)    
    feature_importances.to_csv('ICH-NIHSS-FI-ALL-{}-{}-{}.csv'.format(o_str,w_str,scaler))
    
    fig.savefig(file_str)

    Y = data_nihss_all['NIHSS_group']
    
    if (outliers == True):
        X.drop(index = [1123, 1259, 1256, 1264, 1265, 1260, 1229, 1146, 1187, 1182, 1120,1249], inplace = True)
        Y.drop(index = [1123, 1259, 1256, 1264, 1265, 1260, 1229, 1146, 1187, 1182, 1120,1249], inplace = True)    

    cat_col = []

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, X.columns),
    ])


    models = [
        LogisticRegression(solver='lbfgs'), 
    #    LogisticRegressionCV(solver='lbfgs'),
        RandomForestClassifier(n_estimators=300),
        ExtraTreesClassifier(n_estimators=300),
        AdaBoostClassifier(n_estimators=300),
        SVC(kernel="linear", C=0.025, probability = True)    
    ]

    print(X.columns)
    #random_state = np.random.RandomState(0)
    X = preprocessor.fit_transform(X)
    Y = np.array(Y)
    cv = StratifiedKFold(n_splits=10,random_state=rnd)
          
    file_str = 'ICH-NIHSS-ROC-ALL-{}-{}-{}.png'.format(o_str,w_str,scaler)    
        
    draw_cv_roc_curve(models, cv, X, Y, title='ROC for NIHSS Group Prediction (all predictors)',filename = file_str, weights = weights)

In [ ]:
print('STANDARD SCALER, no weights')
df_all_nihss(data_nihss, 'standard', weights  = False, outliers  = False)
print('\n')

print('STANDARD SCALER, weights')
df_all_nihss(data_nihss, 'standard', weights  = True, outliers  = False)
print('\n')

print('STANDARD SCALER, no outliers')
df_all_nihss(data_nihss, 'standard', weights  = False, outliers  = True)
print('\n')

print('STANDARD SCALER, weights, no outliers')
df_all_nihss(data_nihss, 'standard', weights  = True, outliers  = True)
print('\n')

print('MINMAX SCALER, no weights')
df_all_nihss(data_nihss, 'minmax', weights  = False, outliers  = False)
print('\n')

print('MINMAX SCALER, weights')
df_all_nihss(data_nihss, 'minmax', weights  = True, outliers  = False)
print('\n')

print('MINMAX SCALER, no outliers')
df_all_nihss(data_nihss, 'minmax', weights  = False, outliers  = True)
print('\n')

print('MINMAX SCALER, weights, no outliers')
df_all_nihss(data_nihss, 'minmax', weights  = True, outliers  = True)
print('\n')

In [ ]:
#######################################################################
# 3.PROGNOSTICATION OF HS OUTCOMES (mRS group) FROM INDIVIDUAL RISKS  #
#######################################################################

In [ ]:
data_mrs = data[~data.Discharge_Plan_Modified_Rankin_Score.isna()] #data.copy()
ind = data_mrs[data_mrs.NIHSS_group.isna()].index
data_mrs.drop(index = ind, inplace = True)
ind = data_mrs[data_mrs.Discharge_Plan_Modified_Rankin_Score==11].index
data_mrs.drop(index = ind, inplace = True)

data_mrs.Discharge_Plan_Modified_Rankin_Score.value_counts()

data_mrs['NIHSS_group'] = np.where(data_mrs['Screening_tools_NIHSS']<=4,0,1)
data_mrs['NIHSS_group'].value_counts()

In [ ]:
data_mrs['MRS_group'] = np.where(data_mrs['Discharge_Plan_Modified_Rankin_Score']<=3,0,1)
data_mrs['MRS_group'].value_counts()

In [ ]:
def build_barplot_mrs(df,feature,plot_title,xlabel):
    
    fig = plt.figure(figsize=(8,8))

    g = sns.countplot(df[feature].replace({0:'No',1:'Yes'}), hue = df['MRS_group'])
    plt.title(plot_title)
    plt.xlabel(xlabel)
    plt.ylabel('Counts')

    leg = g.axes.get_legend()
    new_title = 'MRS score'
    leg.set_title(new_title)
    new_labels = ['Less or equal to 3', 'More than 3']
    for t, l in zip(leg.texts, new_labels): t.set_text(l)

    for p in g.patches:
        g.annotate('{:.0f}'.format(p.get_height()), (p.get_x()+0.15, p.get_height()+0.5))

    fig.show()
    fig.savefig('ICH-MRS_{}_barplot.png'.format(feature))

In [ ]:
build_barplot_mrs(data_mrs,'History_OldStroke','MRS group vs Old Stroke','Old Stroke')
build_barplot_mrs(data_mrs,'History_DM','MRS group vs Diabetes Mellitus','Diabetes Mellitus')
build_barplot_mrs(data_mrs,'History_HyperTension','MRS group vs Hyper Tension','Hyper Tension')
build_barplot_mrs(data_mrs,'History_IschemicHeartDisease','MRS group vs Ischemic Heart Disease','Ischemic Heart Disease')
build_barplot_mrs(data_mrs,'History_ArterFibrillation','MRS group vs Arter Fibrillation','Arter Fibrillation')
build_barplot_mrs(data_mrs,'History_HyperLypidAemia','MRS group vs Hyper Lypid Aemia','Hyper Lypid Aemia')

fig = plt.figure(figsize=(8,8))

g = sns.countplot(data_mrs['Day_Time'].replace({1:'Evening',2:'Afternoon',3:'Morning',4:'Night'}), hue = data_mrs['MRS_group'])
plt.title('MRS vs Day Time')
plt.xlabel('History: Day Time')
plt.ylabel('Counts')

leg = g.axes.get_legend()
new_title = 'MRS score'
leg.set_title(new_title)
new_labels = ['Less or equal to 3', 'More than 3']
for t, l in zip(leg.texts, new_labels): t.set_text(l)
     
for p in g.patches:
    g.annotate('{:.0f}'.format(p.get_height()), (p.get_x()+0.15, p.get_height()+0.2))
    
fig.show()
fig.savefig('ICH-MRS_DAY_TIME_barplot.png')

In [ ]:
non_weather_cols = data_mrs.columns.difference(weather_cols)
data_mrs_nw = data_mrs[non_weather_cols]

In [ ]:
def df_cat_mrs(df, scaler, outliers=False, weights=False):

    X = data_mrs_nw.drop(columns=['NIHSS_group','MRS_group','Discharge_Plan_Modified_Rankin_Score'])
    Y = data_mrs_nw['MRS_group']
    
    
    if (outliers == True):
        X.drop(index = [1125, 1150, 1153, 1182, 1196, 1255, 1263, 1275], inplace = True)
        Y.drop(index = [1125, 1150, 1153, 1182, 1196, 1255, 1263, 1275], inplace = True)    
    
    cat_col = ['DEMOGRAPHY_sex','History_ArterFibrillation',
               'History_DM','History_HyperLypidAemia','History_HyperTension','History_IschemicHeartDisease',
               'History_OldStroke','History_Smoking']

    num_col = X.columns.difference(cat_col)
    
    if scaler == 'standard':
        sclr = StandardScaler()
    else:
        sclr = MinMaxScaler()
        
        
    numerical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', sclr),
    ])

    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(drop='first')),
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, num_col),
            ('cat', categorical_transformer, cat_col)
    ])
    print(cat_col)
    print(X.columns)
    
    ada = AdaBoostClassifier()
    gb = GradientBoostingClassifier()
    rf = RandomForestClassifier()
    et = ExtraTreesClassifier()
    
    X_values = preprocessor.fit_transform(X)
    
    onehot_columns = list(preprocessor.transformers_[1][1]['onehot'].get_feature_names(cat_col))
    numeric_features_list = list(num_col)
    numeric_features_list.extend(onehot_columns)
        
    df_from_array_pipeline = pd.DataFrame(X_values, columns = numeric_features_list)
    
    ada.fit(X_values,Y)
    gb.fit(X_values,Y)
    rf.fit(X_values,Y)
    et.fit(X_values,Y)

    df_ada = pd.DataFrame({'feature':np.array(numeric_features_list), 'importance' : ada.feature_importances_})
    df_gb = pd.DataFrame({'feature':np.array(numeric_features_list), 'importance' : gb.feature_importances_})
    df_rf = pd.DataFrame({'feature':np.array(numeric_features_list), 'importance' : rf.feature_importances_})
    df_et = pd.DataFrame({'feature':np.array(numeric_features_list), 'importance' : et.feature_importances_})
    
    result = pd.merge(df_ada[['feature','importance']], df_gb[['feature','importance']], on='feature')
    result = pd.merge(result, df_rf[['feature','importance']], on='feature')
    result = pd.merge(result, df_et[['feature','importance']], on='feature')
    result['importance'] = result.mean(axis=1)
    result = result.sort_values(by='importance', ascending = False)
    feature_importances = result[['feature','importance']]
    print(feature_importances.to_latex())
    
    fig = plt.figure(figsize=(15,6))

    g = sns.barplot(x='feature',y='importance',data=feature_importances, palette = 'tab10')
    g.set_xticklabels(rotation=90,labels = feature_importances['feature'])
    g.set_title('Non-weather feature importances for MRS group')
    plt.show()
    
            
    if (weights == True):
        w_str = 'weights'
    else:
        w_str = 'NOweights'
        
    if (outliers == True):
        o_str = 'out'
    else:
        o_str = 'NOout'
        
    file_str = 'ICH-MRS-FI-NW-{}-{}-{}.png'.format(o_str,w_str,scaler)    
    
    fig.savefig(file_str)
    feature_importances.to_csv('ICH-MRS-FI-NW-{}-{}-{}.csv'.format(o_str,w_str,scaler))

    to_keep = feature_importances.feature[:3]#['BMI', 'DEMOGRAPHY_age', 'Day_Time']
    
  #  print(X.shape, Y.shape)
    
    X = data_mrs_nw[to_keep]
    Y = data_mrs_nw['MRS_group']
  #  print(X.shape, Y.shape)
    
    if (outliers == True):
        X.drop(index = [1125, 1150, 1153, 1182, 1196, 1255, 1263, 1275], inplace = True)
        Y.drop(index = [1125, 1150, 1153, 1182, 1196, 1255, 1263, 1275], inplace = True)    
        
    cat_col = []

    num_col = X.columns.difference(cat_col)

    numerical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', sclr),
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, num_col),
    ])


    models = [
        LogisticRegression(solver='lbfgs'), 
        RandomForestClassifier(n_estimators=300),
        ExtraTreesClassifier(n_estimators=300),
        AdaBoostClassifier(n_estimators=300),
        SVC(kernel="linear", C=0.025, probability = True)    
    ]

    X = preprocessor.fit_transform(X)
    Y = np.array(Y)
    cv = StratifiedKFold(n_splits=10,random_state=rnd)
    print(X.shape, Y.shape)
    if (weights == True):
        w_str = 'weights'
    else:
        w_str = 'NOweights'
        
    if (outliers == True):
        o_str = 'out'
    else:
        o_str = 'NOout'
        
    file_str = 'MRS-ROC-NW-{}-{}-{}.png'.format(o_str,w_str,scaler)    
    print(X.shape, Y.shape)    
    draw_cv_roc_curve(models, cv, X, Y, title='ROC for MRS Group Prediction (non-weather predictors)',filename = file_str, weights = weights)

In [ ]:
print('STANDARD SCALER, no weights')
df_cat_mrs(data_mrs_nw, 'standard', weights  = False, outliers  = False)
print('\n')

print('STANDARD SCALER, weights')
df_cat_mrs(data_mrs_nw, 'standard', weights  = True, outliers  = False)
print('\n')

print('STANDARD SCALER, no outliers')
df_cat_mrs(data_mrs_nw, 'standard', weights  = False, outliers  = True)
print('\n')

print('STANDARD SCALER, weights, no outliers')
df_cat_mrs(data_mrs_nw, 'standard', weights  = True, outliers  = True)
print('\n')

print('MINMAX SCALER, no weights')
df_cat_mrs(data_mrs_nw, 'minmax', weights  = False, outliers  = False)
print('\n')

print('MINMAX SCALER, weights')
df_cat_mrs(data_mrs_nw, 'minmax', weights  = True, outliers  = False)
print('\n')

print('MINMAX SCALER, no outliers')
df_cat_mrs(data_mrs_nw, 'minmax', weights  = False, outliers  = True)
print('\n')

print('MINMAX SCALER, weights, no outliers')
df_cat_mrs(data_mrs_nw, 'minmax', weights  = True, outliers  = True)
print('\n')

In [ ]:
################################################################################
# 4.PORGNOSTICATION OF HS OUTCOMES (mRS group) FROM THE FULL SET OF PREDICTORS #
# (BOTH INDIVIDUAL RISKS AND WEATHER PARAMETERS)                               #
################################################################################

In [ ]:
def df_all_mrs(df, scaler, outliers=False, weights=False):
    data_mrs_all = data_mrs[['DEMOGRAPHY_age', 'BMI', 'TEMP', 'STP', 'WDSP', 'RH', 'HUMIDEX','Day_Time',
       'TEMP1', 'TEMP2', 'STP1', 'STP2', 'WDSP1', 'WDSP2', 'RH1', 'RH2','Screening_tools_NIHSS',
       'HUMIDEX1', 'HUMIDEX2', 'TDIF1', 'TDIF2', 'PDIF1', 'PDIF2', 'WDIF1',
       'WDIF2', 'RHDIF1', 'RHDIF2', 'HDIF1', 'HDIF2', 'MRS_group']]
    cor_matrix = data_mrs_all.corr().abs()
    upper_tri = cor_matrix.where(np.triu(np.ones(cor_matrix.shape),k=1).astype(np.bool))
    to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.95)]
    data_mrs_all = data_mrs_all.drop(columns = to_drop)
    
    X = data_mrs_all.drop(columns=['MRS_group'])
    Y = data_mrs_all['MRS_group']

    if (outliers == True):
        X.drop(index = [1123, 1259, 1256, 1264, 1265, 1260, 1229, 1146, 1187, 1182, 1120,1249], inplace = True)
        Y.drop(index = [1123, 1259, 1256, 1264, 1265, 1260, 1229, 1146, 1187, 1182, 1120,1249], inplace = True)    

    cat_col = []

    num_col = X.columns.difference(cat_col)

    if scaler == 'standard':
        sclr = StandardScaler()
    else:
        sclr = MinMaxScaler()

    
    numerical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', sclr),
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, num_col),
    ])

    ada = AdaBoostClassifier()
    gb = GradientBoostingClassifier()
    rf = RandomForestClassifier()
    et = ExtraTreesClassifier()
    
    X_values = preprocessor.fit_transform(X)
    
    numeric_features_list = list(num_col)
    
    df_from_array_pipeline = pd.DataFrame(X_values, columns = numeric_features_list)
    
    ada.fit(X_values,Y)
    gb.fit(X_values,Y)
    rf.fit(X_values,Y)
    et.fit(X_values,Y)

    df_ada = pd.DataFrame({'feature':np.array(numeric_features_list), 'importance' : ada.feature_importances_})
    df_gb = pd.DataFrame({'feature':np.array(numeric_features_list), 'importance' : gb.feature_importances_})
    df_rf = pd.DataFrame({'feature':np.array(numeric_features_list), 'importance' : rf.feature_importances_})
    df_et = pd.DataFrame({'feature':np.array(numeric_features_list), 'importance' : et.feature_importances_})
    
    result = pd.merge(df_ada[['feature','importance']], df_gb[['feature','importance']], on='feature')
    result = pd.merge(result, df_rf[['feature','importance']], on='feature')
    result = pd.merge(result, df_et[['feature','importance']], on='feature')
    result['importance'] = result.mean(axis=1)
    result = result.sort_values(by='importance', ascending = False)
    feature_importances = result[['feature','importance']]
    print(feature_importances.to_latex())
    to_keep = list(feature_importances[feature_importances['importance']>0.04]['feature'])
    print('Features to keep:', to_keep)
    X = data_mrs_all[to_keep]
        
    fig = plt.figure(figsize=(15,6))

    g = sns.barplot(x='feature',y='importance',data=feature_importances, palette = 'tab10')
    g.set_xticklabels(rotation=90,labels = feature_importances['feature'])
    g.set_title('All feature importances for MRS group')
    plt.show()
    
        
    if (weights == True):
        w_str = 'weights'
    else:
        w_str = 'NOweights'
        
    if (outliers == True):
        o_str = 'out'
    else:
        o_str = 'NOout'
        
    file_str = 'ICH-MRS-FI-ALL-{}-{}-{}.png'.format(o_str,w_str,scaler)    
        
    fig.savefig(file_str)
    
    feature_importances.to_csv('ICH-MRS-FI-ALL-{}-{}-{}.csv'.format(o_str,w_str,scaler))


    Y = data_mrs_all['MRS_group']
    
    if (outliers == True):
        X.drop(index = [1123, 1259, 1256, 1264, 1265, 1260, 1229, 1146, 1187, 1182, 1120,1249], inplace = True)
        Y.drop(index = [1123, 1259, 1256, 1264, 1265, 1260, 1229, 1146, 1187, 1182, 1120,1249], inplace = True)    

    cat_col = []

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, X.columns),
    ])


    models = [
        LogisticRegression(solver='lbfgs'), 
    #    LogisticRegressionCV(solver='lbfgs'),
        RandomForestClassifier(n_estimators=300),
        ExtraTreesClassifier(n_estimators=300),
        AdaBoostClassifier(n_estimators=300),
        SVC(kernel="linear", C=0.025, probability = True)    
    ]

    print(X.columns)
    X = preprocessor.fit_transform(X)
    Y = np.array(Y)
    cv = StratifiedKFold(n_splits=10,random_state=rnd)
          
    file_str = 'ICH-MRS-ROC-ALL-{}-{}-{}.png'.format(o_str,w_str,scaler)    
        
   
    to_keep = list(feature_importances[feature_importances['importance']>0.04]['feature'])

    fig1 = plt.figure(figsize=[12,12])
    ax1 = fig1.add_subplot(111,aspect = 'equal')

    model = Sequential()
    model.add(Dense(12, input_dim = 1, activation = 'relu'))
    model.add(Dense(10, activation = 'relu'))
    model.add(Dense(1, activation = 'sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer = 'adam')        
    
    data_tmp = data_mrs.copy()
    if (outliers == True):
        data_tmp.drop(index = [1123, 1259, 1256, 1264, 1265, 1260, 1229, 1146, 1187, 1182, 1120,1249], inplace = True)
    
    df_roc = pd.DataFrame([])
        
    for feature in to_keep:
        X = data_tmp[feature]
        Y = data_tmp['MRS_group']

        numerical_transformer = Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', sclr),
        ])

        X = np.array(X)

        X = numerical_transformer.fit_transform(X.reshape(-1, 1))
        Y = np.array(Y)

        cv = StratifiedKFold(n_splits=10)
        tprs = []
        aucs = []


        class_weights = class_weight.compute_class_weight(
               'balanced',
                np.unique(Y), 
                Y)

        class_dict = {0:class_weights[0],1:class_weights[1]} 
     
        mean_fpr = np.linspace(0,1,100)
        i = 1
        for train, test in cv.split(X,Y):
            if (weights == True):
                model.fit(X[train],Y[train],validation_data=(X[test],Y[test]),class_weight=class_dict, verbose=0,epochs=500)
            else:
                model.fit(X[train],Y[train],validation_data=(X[test],Y[test]),verbose=0,epochs=500)

            prediction = model.predict(X[test])

            fpr, tpr, t = roc_curve(Y[test], prediction)
            tprs.append(interp(mean_fpr, fpr, tpr))
            roc_auc = auc(fpr, tpr)
            aucs.append(roc_auc)
            i= i+1

        plt.plot([0,1],[0,1],linestyle = '--',lw = 2,color = 'black')
        mean_tpr = np.mean(tprs, axis=0)
        mean_auc = auc(mean_fpr, mean_tpr)
        plt.plot(mean_fpr, mean_tpr, color=np.random.rand(3,),
                 label=r'{} (AUC = {:.3} )'.format(feature,mean_auc),lw=2, alpha=1)
        df_roc = df_roc.append(pd.Series([feature,mean_auc]),ignore_index = True)
    
    df_roc.columns = ['Feature','ROC']
    
    df_roc.sort_values(by=['ROC'], inplace = True)
    
    print(df_roc.to_latex())
    
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC curves for MRS prediction')
    plt.legend(loc="lower right")
    plt.show()
    
    if (weights == True):
        w_str = 'weights'
    else:
        w_str = 'NOweights'
        
    if (outliers == True):
        o_str = 'out'
    else:
        o_str = 'NOout'
        
    file_str = 'ICH-ROC-MRS-all-{}-{}-{}.png'.format(o_str,w_str,scaler)    
        
    df_roc.to_csv('ICH-ROC-MRS-all-ROC-{}-{}-{}.csv')
    fig1.savefig(file_str)
    
    selected_features = df_roc.Feature[:3]

    all_features = to_keep
    print('Selected features:',selected_features)
    print('All features:',all_features)

    fig1 = plt.figure(figsize=[12,12])
    ax1 = fig1.add_subplot(111,aspect = 'equal')



    for feature in [selected_features, all_features]:
        X = data_tmp[feature]
        Y = data_tmp['MRS_group']

        model = Sequential()
        model.add(Dense(12, input_dim = X.shape[1], activation = 'relu'))
        model.add(Dense(10, activation = 'relu'))
        model.add(Dense(1, activation = 'sigmoid'))
        model.compile(loss='binary_crossentropy', optimizer = 'adam')        

        cv = StratifiedKFold(n_splits=10)
        tprs = []
        aucs = []

        numerical_transformer = Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', sclr),
        ])

        X = np.array(X)

        X = numerical_transformer.fit_transform(X)
        Y = np.array(Y)

        mean_fpr = np.linspace(0,1,100)
        i = 1
        for train, test in cv.split(X,Y):
            
            if (weights == True):
                model.fit(X[train],Y[train],validation_data=(X[test],Y[test]),class_weight=class_dict, verbose=0,epochs=500)
            else:
                model.fit(X[train],Y[train],validation_data=(X[test],Y[test]),verbose=0,epochs=500)

            prediction = model.predict(X[test])

            fpr, tpr, t = roc_curve(Y[test], prediction)
            tprs.append(interp(mean_fpr, fpr, tpr))
            roc_auc = auc(fpr, tpr)
            aucs.append(roc_auc)
            i= i+1

        plt.plot([0,1],[0,1],linestyle = '--',lw = 2,color = 'black')
        mean_tpr = np.mean(tprs, axis=0)
        mean_auc = auc(mean_fpr, mean_tpr)

        if (len(X[0])==3):
            label = r'{} (AUC = {:.3} )'.format(selected_features, mean_auc)
        else:
            label = r'All Significant Features (AUC = {:.3} )'.format(mean_auc)

        plt.plot(mean_fpr, mean_tpr, color=np.random.rand(3,),
                 label=label,lw=2, alpha=1)

    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC curves for MRS prediction')
    plt.legend(loc="lower right")
    plt.show()
    
    if (weights == True):
        w_str = 'weights'
    else:
        w_str = 'NOweights'
        
    if (outliers == True):
        o_str = 'out'
    else:
        o_str = 'NOout'
        
    file_str = 'ICH-ROC-MRS-2-{}-{}-{}.png'.format(o_str,w_str,scaler)    
    
    fig1.savefig(file_str)

In [ ]:
print('STANDARD SCALER, no weights')
df_all_mrs(data_mrs, 'standard', weights  = False, outliers  = False)
print('\n')

print('STANDARD SCALER, weights')
df_all_mrs(data_mrs, 'standard', weights  = True, outliers  = False)
print('\n')

print('STANDARD SCALER, no outliers')
df_all_mrs(data_mrs, 'standard', weights  = False, outliers  = True)
print('\n')

print('STANDARD SCALER, weights, no outliers')
df_all_mrs(data_mrs, 'standard', weights  = True, outliers  = True)
print('\n')

print('MINMAX SCALER, no weights')
df_all_mrs(data_mrs, 'minmax', weights  = False, outliers  = False)
print('\n')

print('MINMAX SCALER, weights')
df_all_mrs(data_mrs, 'minmax', weights  = True, outliers  = False)
print('\n')

print('MINMAX SCALER, no outliers')
df_all_mrs(data_mrs, 'minmax', weights  = False, outliers  = True)
print('\n')

print('MINMAX SCALER, weights, no outliers')
df_all_mrs(data_mrs, 'minmax', weights  = True, outliers  = True)
print('\n')